# Dependencies

In [ ]:
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install bs4

import time
import pandas

In [5]:
from apis import Genius, Spotify

# Scraping

In [6]:
genius = Genius()
genius.authenticate()

Please follow this link and authorize the app:
https://api.genius.com/oauth/authorize?client_id=HDGdy8grlfzttb5mZx3hKJzLdgJdlTnyxGTPY06Ldkrx21MCkug1Vi6UQwgW4KXW&redirect_uri=http%3A%2F%2Flocalhost&scope=&state=&response_type=code

Then, paste the URL you are redirected to here: 
http://localhost/?code=3PD6vYoA95z6AScnTgbMerBvjFebh4g4QRSsGTIglMkdMB2XhNx6aQUZ13SX3ryQ
Authentication successful.


First, we gather Genius artist IDs for a selection of 125 artists (Billboard "Greatest of All Time").

In [7]:
# Billboard "Greatest of All Time Artists"
# Source: https://www.billboard.com/charts/greatest-of-all-time-artists
artists = ["The Beatles","The Rolling Stones","Elton John","Mariah Carey","Madonna","Barbra Streisand","Michael Jackson","Taylor Swift","Stevie Wonder","Chicago","Whitney Houston","Paul McCartney","Elvis Presley","Janet Jackson","Rod Stewart","Drake","Prince","Rihanna","Billy Joel","Garth Brooks","Herb Alpert","Eminem","Usher","Bruce Springsteen","Neil Diamond","The Supremes","Eagles","Bee Gees","The Beach Boys","Fleetwood Mac","Santana","The Temptations","Celine Dion","Aretha Franklin","Daryl Hall John Oates","Adele","Beyonce","Eric Clapton","Lionel Richie","Led Zeppelin","Bruno Mars","John Denver","Maroon 5","Olivia Newton-John","Bon Jovi","Phil Collins","Diana Ross","Bob Dylan","John Mellencamp","JAY-Z","Donna Summer","Nickelback","Boyz II Men","Carpenters","Justin Bieber","Lady Gaga","Foreigner","Britney Spears","P!nk","Alicia Keys","Katy Perry","Miley Cyrus","The Jacksons","Earth, Wind & Fire","Marvin Gaye","U2","Linda Ronstadt","Van Halen","Bob Seger","Aerosmith","The Black Eyed Peas","Nelly","Carole King","Justin Timberlake","The Monkees","Kenny Rogers","Heart","Tim McGraw","Barry Manilow","R. Kelly","Journey","Simon & Garfunkel","Michael Bolton","Ed Sheeran","Kelly Clarkson","Pink Floyd","Queen","Mary J. Blige","Kanye West","Three Dog Night","Commodores","Paula Abdul","Bryan Adams","Chris Brown","Guns N' Roses","Def Leppard","Huey Lewis & The News","Linkin Park","James Taylor","Creedence Clearwater Revival","Tom Petty And The Heartbreakers","Paul Simon","Backstreet Boys","Lil Wayne","Carrie Underwood","Andy Williams","Styx","George Michael","Cher","TLC","Shania Twain","The Doobie Brothers","The 4 Seasons","Peter, Paul & Mary","Kenny Chesney","Destiny's Child","50 Cent","Ray Charles","Genesis","The Police","Rascal Flatts","Ludacris","Gloria Estefan","George Harrison","Alan Jackson"]

In [8]:
artist_ids = [None] * len(artists)
# Get Genius ID for each artist
for i, artist in enumerate(artists):
    res = genius.get("search", q=artist)
    if "response" not in res or "hits" not in res["response"]:
        print("Invalid response for artist {}".format(artist))
        continue
    hits = res["response"]["hits"]
    if not hits:
        print("No results for artist {}".format(artist))
        continue
    result_artist_ids = []
    for hit in hits:
        if not "type" in hit or hit["type"] != "song":
            continue
        if not "result" in hit or not "primary_artist" in hit["result"] or not "id" in hit["result"]["primary_artist"]:
            continue
        result_artist_ids.append(hit["result"]["primary_artist"]["id"])
    if not result_artist_ids:
        print("Found no artist ID for artist {}".format(artist))
    else:
        # Get artist ID that appears the most often in results (on ties, pick first)
        artist_id_counts = list(map(result_artist_ids.count, result_artist_ids))
        artist_id = [gid for gid, cnt in zip(result_artist_ids, artist_id_counts) if cnt == max(artist_id_counts)][0]
        print("{}: {} -> {}".format(i, artist, artist_id))
    artist_ids[i] = artist_id

0: The Beatles -> 586
1: The Rolling Stones -> 774
2: Elton John -> 560
3: Mariah Carey -> 1205
4: Madonna -> 276
5: Barbra Streisand -> 65306
6: Michael Jackson -> 835
7: Taylor Swift -> 1177
8: Stevie Wonder -> 1602
9: Chicago -> 1955
10: Whitney Houston -> 8534
11: Paul McCartney -> 1601
12: Elvis Presley -> 1645
13: Janet Jackson -> 2122
14: Rod Stewart -> 97739
15: Drake -> 130
16: Prince -> 1630
17: Rihanna -> 89
18: Billy Joel -> 16965
19: Garth Brooks -> 15266
20: Herb Alpert -> 147200
21: Eminem -> 45
22: Usher -> 132
23: Bruce Springsteen -> 19041
24: Neil Diamond -> 44735
25: The Supremes -> 100713
26: Eagles -> 561
27: Bee Gees -> 800
28: The Beach Boys -> 1138
29: Fleetwood Mac -> 2579
30: Santana -> 8439
31: The Temptations -> 1664
32: Celine Dion -> 65300
33: Aretha Franklin -> 1108
34: Daryl Hall John Oates -> 49926
35: Adele -> 2300
36: Beyonce -> 498
37: Eric Clapton -> 19272
38: Lionel Richie -> 22272
39: Led Zeppelin -> 562
40: Bruno Mars -> 500
41: John Denver -> 1

In [73]:
for i, (artist, artist_id) in enumerate(zip(artists, artist_ids)):
    for j, (artist_, artist_id_) in enumerate(zip(artists, artist_ids)):
        if i == j:
            continue
        if artist_id == artist_id_:
            print("Duplicate artist {}, {}: {} ({}), {} ({})".format(i, j, artist, artist_, artist_id, artist_id_))

In [72]:
# Manually fix wrong artist IDs.
def set_artist_id(artist, gid):
    for i, (this_artist, this_gid) in enumerate(zip(artists, artist_ids)):
        if this_artist != artist:
            continue
        artist_ids[i] = gid

set_artist_id("Heart", 7760)
set_artist_id("Queen", 563)

Gather fifty songs for each artist.

In [81]:
dataset = []  # list of dicts {"genius_id": ..., "title": ..., "path": ..., "spotify_uri": ..., "lyrics": ...,
              #                 "artist": ..., "artist_genius_id": ..., spotify features}
dataset = [{"genius_id": 1}, {"genius_id": 2}, {"genius_id": 1}]

In [85]:
for i, (artist, artist_id) in enumerate(zip(artists, artist_ids)):
    try:
        songs_response = genius.get("artists/{}/songs".format(artist_id), per_page=50, sort="popularity")
        if not "response" in songs_response or not "songs" in songs_response["response"]:
            print("Error gathering songs for artist {}. {}".format(artist, str(songs_response)))
            continue
        songs = songs_response["response"]["songs"]
        print("({0:3d}/{1:3d}) Gathered {2:2d} songs for artist {3:s}.".format(i, len(artists), len(songs), artist))
        for song in songs:
            if not "id" in song or not "path" in song or not "title" in song:
                print("Error for song. {}".format(str(song)))
                continue
            data = {"genius_id" : song["id"],
                    "title": song["title"], "path": song["path"],
                    "artist": artist, "artist_genius_id": artist_id,
                    "spotify_uri": None, "lyrics": None}
            dataset.append(data)
        print("          Including most popular song {}.".format(songs[0]["title"]))
    except Exception or KeyboardInterrupt as e:
        print("Error for song request {}. {}".format(i, repr(e)))
        break

(  0/125) Gathered 50 songs for artist The Beatles.
          Including most popular song Let It Be.
(  1/125) Gathered 50 songs for artist The Rolling Stones.
          Including most popular song Sympathy for the Devil.
(  2/125) Gathered 50 songs for artist Elton John.
          Including most popular song All of the Lights.
(  3/125) Gathered 50 songs for artist Mariah Carey.
          Including most popular song Don’t.
(  4/125) Gathered 50 songs for artist Madonna.
          Including most popular song Like a Prayer.
(  5/125) Gathered 50 songs for artist Barbra Streisand.
          Including most popular song We Are the World 25 for Haiti.
(  6/125) Gathered 50 songs for artist Michael Jackson.
          Including most popular song King Kunta.
(  7/125) Gathered 50 songs for artist Taylor Swift.
          Including most popular song ​cardigan.
(  8/125) Gathered 50 songs for artist Stevie Wonder.
          Including most popular song Doing It Wrong.
(  9/125) Gathered 50 songs f

( 78/125) Gathered 50 songs for artist Barry Manilow.
          Including most popular song Copacabana (At the Copa).
( 79/125) Gathered 50 songs for artist R. Kelly.
          Including most popular song Do What U Want.
( 80/125) Gathered 50 songs for artist Journey.
          Including most popular song Don’t Stop Believin’.
( 81/125) Gathered 50 songs for artist Simon & Garfunkel.
          Including most popular song The Sound of Silence.
( 82/125) Gathered 50 songs for artist Michael Bolton.
          Including most popular song Never Let Me Down.
( 83/125) Gathered 50 songs for artist Ed Sheeran.
          Including most popular song Shape of You.
( 84/125) Gathered 50 songs for artist Kelly Clarkson.
          Including most popular song Because of You.
( 85/125) Gathered 49 songs for artist Pink Floyd.
          Including most popular song Wish You Were Here.
( 86/125) Gathered 50 songs for artist Queen.
          Including most popular song Bohemian Rhapsody.
( 87/125) Gathere

In [86]:
# Prune duplicates.
old_dataset = dataset.copy()
ids = {row["genius_id"] for row in dataset}
before_len = len(dataset)
for gid in ids:
    to_delete = []
    matches = 0
    for i, row in enumerate(dataset):
        if row["genius_id"] == gid:
            matches += 1
            if matches > 1:
                to_delete.append(i)
    for i in to_delete:
        del dataset[i]
after_len = len(dataset)
print("Pruned {} duplicates.".format(before_len-after_len))

Pruned 313 duplicates.


In [95]:
# Get spotify IDs for each song
for i, row in enumerate(dataset):
    try:
        song_response = genius.get("songs/{}".format(row["genius_id"]))
        if not "response" in song_response or not "song" in song_response["response"]:
            print("Error")
            continue
        song = song_response["response"]["song"]
        print("({}/{}) Querying Spotify ID for song {}.".format(i, len(dataset), row["title"]))
        if not "media" in song:
            print("No media.")
            continue
        spotify_uri = None
        for media in song["media"]:
            if not "provider" in media or not "native_uri" in media:
                continue
            if media["provider"] == "spotify":
                spotify_uri = media["native_uri"]
                break
        else:
            print("No spotify ID.")
        dataset[i]["spotify_uri"] = spotify_uri
    except Exception or KeyboardInterrupt as e:
        print("Error getting Spotify ID in request {}. {}".format(i, repr(e)))
        break

(0/5862) Querying Spotify ID for song Let It Be.
(1/5862) Querying Spotify ID for song Yesterday.
(2/5862) Querying Spotify ID for song Come Together.
(3/5862) Querying Spotify ID for song Hey Jude.
(4/5862) Querying Spotify ID for song Something.
(5/5862) Querying Spotify ID for song Here Comes the Sun.
(6/5862) Querying Spotify ID for song A Day in the Life.
(7/5862) Querying Spotify ID for song Blackbird.
(8/5862) Querying Spotify ID for song I Am the Walrus.
(9/5862) Querying Spotify ID for song In My Life.
(10/5862) Querying Spotify ID for song Eleanor Rigby.
(11/5862) Querying Spotify ID for song While My Guitar Gently Weeps.
(12/5862) Querying Spotify ID for song Lucy in the Sky with Diamonds.
(13/5862) Querying Spotify ID for song Across the Universe.
(14/5862) Querying Spotify ID for song Strawberry Fields Forever.
(15/5862) Querying Spotify ID for song Michelle.
(16/5862) Querying Spotify ID for song With a Little Help from My Friends.
(17/5862) Querying Spotify ID for song H

(134/5862) Querying Spotify ID for song Fairweather Friends.
(135/5862) Querying Spotify ID for song Ordinary Man.
No spotify ID.
(136/5862) Querying Spotify ID for song Sacrifice.
(137/5862) Querying Spotify ID for song That’s What Friends Are For.
No spotify ID.
(138/5862) Querying Spotify ID for song Circle of Life (Broadway Version).
No spotify ID.
(139/5862) Querying Spotify ID for song Levon.
(140/5862) Querying Spotify ID for song I’m Still Standing.
No spotify ID.
(141/5862) Querying Spotify ID for song Wonderful Life.
(142/5862) Querying Spotify ID for song Mona Lisas and Mad Hatters.
(143/5862) Querying Spotify ID for song Someone Saved My Life Tonight.
No spotify ID.
(144/5862) Querying Spotify ID for song The One.
No spotify ID.
(145/5862) Querying Spotify ID for song Honky Cat.
No spotify ID.
(146/5862) Querying Spotify ID for song Border Song.
No spotify ID.
(147/5862) Querying Spotify ID for song Elephant Love Medley.
No spotify ID.
(148/5862) Querying Spotify ID for son

(267/5862) Querying Spotify ID for song You Are Woman, I Am Man.
No spotify ID.
(268/5862) Querying Spotify ID for song If I Could.
No spotify ID.
(269/5862) Querying Spotify ID for song Papa, Can You Hear Me?.
No spotify ID.
(270/5862) Querying Spotify ID for song People.
No spotify ID.
(271/5862) Querying Spotify ID for song A Piece of Sky.
No spotify ID.
(272/5862) Querying Spotify ID for song I Finally Found Someone.
No spotify ID.
(273/5862) Querying Spotify ID for song What Kind of Fool.
No spotify ID.
(274/5862) Querying Spotify ID for song I Still Can See Your Face.
No spotify ID.
(275/5862) Querying Spotify ID for song Happy Days Are Here Again.
No spotify ID.
(276/5862) Querying Spotify ID for song You Don’t Bring Me Flowers.
No spotify ID.
(277/5862) Querying Spotify ID for song Somewhere (2014).
No spotify ID.
(278/5862) Querying Spotify ID for song I’m Five.
No spotify ID.
(279/5862) Querying Spotify ID for song Put On Your Sunday Clothes.
No spotify ID.
(280/5862) Queryin

(391/5862) Querying Spotify ID for song ​gold rush.
No spotify ID.
(392/5862) Querying Spotify ID for song Bad Blood.
(393/5862) Querying Spotify ID for song Miss Americana & The Heartbreak Prince.
No spotify ID.
(394/5862) Querying Spotify ID for song ​tolerate it.
No spotify ID.
(395/5862) Querying Spotify ID for song London Boy.
No spotify ID.
(396/5862) Querying Spotify ID for song Cornelia Street.
No spotify ID.
(397/5862) Querying Spotify ID for song All Too Well.
(398/5862) Querying Spotify ID for song Wildest Dreams.
(399/5862) Querying Spotify ID for song Doing It Wrong.
(400/5862) Querying Spotify ID for song Gangsta’s Paradise.
(401/5862) Querying Spotify ID for song STOP TRYING TO BE GOD.
No spotify ID.
(402/5862) Querying Spotify ID for song Smile.
No spotify ID.
(403/5862) Querying Spotify ID for song Jorja Interlude.
No spotify ID.
(404/5862) Querying Spotify ID for song So Many Tears.
(405/5862) Querying Spotify ID for song Viajo Sin Ver (Remix).
No spotify ID.
(406/586

(521/5862) Querying Spotify ID for song Queen of the Night.
No spotify ID.
(522/5862) Querying Spotify ID for song I Love the Lord.
(523/5862) Querying Spotify ID for song My Love is Your Love (Remix).
(524/5862) Querying Spotify ID for song Try It on My Own.
No spotify ID.
(525/5862) Querying Spotify ID for song The Star Spangled Banner.
No spotify ID.
(526/5862) Querying Spotify ID for song The Prince Of Egypt (When You Believe).
(527/5862) Querying Spotify ID for song Transcript of ‘Crack is whack’ Interview.
No spotify ID.
(528/5862) Querying Spotify ID for song Step By Step.
No spotify ID.
(529/5862) Querying Spotify ID for song I Go to the Rock.
No spotify ID.
(530/5862) Querying Spotify ID for song I Believe In You and Me (Film Version).
(531/5862) Querying Spotify ID for song Joy to the World.
(532/5862) Querying Spotify ID for song Jesus Loves Me.
No spotify ID.
(533/5862) Querying Spotify ID for song I Wanna Dance With Somebody (Who Loves Me).
No spotify ID.
(534/5862) Queryi

...

(5367/5862) Querying Spotify ID for song Anything But Mine.
(5368/5862) Querying Spotify ID for song Wild Child.
(5369/5862) Querying Spotify ID for song All the Pretty Girls.
No spotify ID.
(5370/5862) Querying Spotify ID for song Come Over.
(5371/5862) Querying Spotify ID for song Everything’s Gonna Be Alright.
No spotify ID.
(5372/5862) Querying Spotify ID for song When the Sun Goes Down.
(5373/5862) Querying Spotify ID for song Setting the World on Fire.
No spotify ID.
(5374/5862) Querying Spotify ID for song The Good Stuff.
(5375/5862) Querying Spotify ID for song Tip of My Tongue.
No spotify ID.
(5376/5862) Querying Spotify ID for song Somewhere With You.
(5377/5862) Querying Spotify ID for song Don’t Blink.
(5378/5862) Querying Spotify ID for song You And Tequila.
No spotify ID.
(5379/5862) Querying Spotify ID for song I Go Back.
(5380/5862) Querying Spotify ID for song There Goes My Life.
No spotify ID.
(5381/5862) Querying Spotify ID for song ​half of my hometown.
No spotify I

(5495/5862) Querying Spotify ID for song Life’s on the Line.
No spotify ID.
(5496/5862) Querying Spotify ID for song No Romeo No Juliet.
No spotify ID.
(5497/5862) Querying Spotify ID for song High All the Time.
(5498/5862) Querying Spotify ID for song Just a Lil’ Bit.
(5499/5862) Querying Spotify ID for song Down on Me.
(5500/5862) Querying Spotify ID for song Encore / Curtains Down.
No spotify ID.
(5501/5862) Querying Spotify ID for song When it Rains it Pours.
(5502/5862) Querying Spotify ID for song Still Be Friends.
No spotify ID.
(5503/5862) Querying Spotify ID for song I’ll Whip Ya Head Boy.
(5504/5862) Querying Spotify ID for song U Not Like Me.
(5505/5862) Querying Spotify ID for song To Love & Die.
(5506/5862) Querying Spotify ID for song Many Men.
No spotify ID.
(5507/5862) Querying Spotify ID for song Hail Mary.
(5508/5862) Querying Spotify ID for song We Are the World.
(5509/5862) Querying Spotify ID for song Hit the Road Jack.
(5510/5862) Querying Spotify ID for song Geor

(5618/5862) Querying Spotify ID for song Murder by Numbers.
No spotify ID.
(5619/5862) Querying Spotify ID for song Bring on the Night.
No spotify ID.
(5620/5862) Querying Spotify ID for song Invisible Sun.
No spotify ID.
(5621/5862) Querying Spotify ID for song Spirits in the Material World.
No spotify ID.
(5622/5862) Querying Spotify ID for song Canary in a Coalmine.
No spotify ID.
(5623/5862) Querying Spotify ID for song Mother.
(5624/5862) Querying Spotify ID for song The Bed’s Too Big Without You.
No spotify ID.
(5625/5862) Querying Spotify ID for song Driven to Tears.
No spotify ID.
(5626/5862) Querying Spotify ID for song Tea in the Sahara.
No spotify ID.
(5627/5862) Querying Spotify ID for song Darkness.
No spotify ID.
(5628/5862) Querying Spotify ID for song Miss Gradenko.
No spotify ID.
(5629/5862) Querying Spotify ID for song Walking in Your Footsteps.
No spotify ID.
(5630/5862) Querying Spotify ID for song Next to You.
No spotify ID.
(5631/5862) Querying Spotify ID for song

(5736/5862) Querying Spotify ID for song Country Shit (Remix).
No spotify ID.
(5737/5862) Querying Spotify ID for song Rollout (My Business).
(5738/5862) Querying Spotify ID for song How Low.
(5739/5862) Querying Spotify ID for song Vitamin D.
(5740/5862) Querying Spotify ID for song Sex Room.
(5741/5862) Querying Spotify ID for song Pimpin’ All Over the World.
(5742/5862) Querying Spotify ID for song Hip Hop Quotables.
(5743/5862) Querying Spotify ID for song Double Cup.
No spotify ID.
(5744/5862) Querying Spotify ID for song Everyday Birthday.
(5745/5862) Querying Spotify ID for song Legs Shakin’.
(5746/5862) Querying Spotify ID for song Almost Like Praying.
No spotify ID.
(5747/5862) Querying Spotify ID for song Whenever, Wherever.
(5748/5862) Querying Spotify ID for song Whenever.
(5749/5862) Querying Spotify ID for song Conga.
(5750/5862) Querying Spotify ID for song Let’s Get Loud.
No spotify ID.
(5751/5862) Querying Spotify ID for song Just Another Day.
No spotify ID.
(5752/5862

(5854/5862) Querying Spotify ID for song Softly And Tenderly.
No spotify ID.
(5855/5862) Querying Spotify ID for song Precious Memories.
No spotify ID.
(5856/5862) Querying Spotify ID for song Sissy’s Song.
No spotify ID.
(5857/5862) Querying Spotify ID for song It Must Be Love.
No spotify ID.
(5858/5862) Querying Spotify ID for song In The Garden.
No spotify ID.
(5859/5862) Querying Spotify ID for song How Great Thou Art.
No spotify ID.
(5860/5862) Querying Spotify ID for song You Can Always Come Home.
(5861/5862) Querying Spotify ID for song Let It Be Christmas.
No spotify ID.


In [103]:
# Get lyrics
for i, row in enumerate(dataset):
    if not row["spotify_uri"]:
        continue  # don't waste time
    path = row["path"]
    try:
        print("({}/{}) Scraping lyrics for song {}.".format(i, len(dataset), row["title"]))
        lyrics = genius.lyrics_from_song_api_path(path)
        dataset[i]["lyrics"] = lyrics
    except Exception as e:
        print("Error for song {}. {}".format(row["title"], e))

(0/5862) Scraping lyrics for song Let It Be.
(1/5862) Scraping lyrics for song Yesterday.
(2/5862) Scraping lyrics for song Come Together.
(3/5862) Scraping lyrics for song Hey Jude.
(4/5862) Scraping lyrics for song Something.
(5/5862) Scraping lyrics for song Here Comes the Sun.
(6/5862) Scraping lyrics for song A Day in the Life.
(7/5862) Scraping lyrics for song Blackbird.
(8/5862) Scraping lyrics for song I Am the Walrus.
(9/5862) Scraping lyrics for song In My Life.
(10/5862) Scraping lyrics for song Eleanor Rigby.
(11/5862) Scraping lyrics for song While My Guitar Gently Weeps.
(12/5862) Scraping lyrics for song Lucy in the Sky with Diamonds.
(13/5862) Scraping lyrics for song Across the Universe.
(14/5862) Scraping lyrics for song Strawberry Fields Forever.
(15/5862) Scraping lyrics for song Michelle.
(16/5862) Scraping lyrics for song With a Little Help from My Friends.
(17/5862) Scraping lyrics for song Help!.
(18/5862) Scraping lyrics for song I Want to Hold Your Hand.
(19/5

(205/5862) Scraping lyrics for song Justify My Thug.
(207/5862) Scraping lyrics for song Bitch I’m Madonna.
(209/5862) Scraping lyrics for song Hung Up.
(211/5862) Scraping lyrics for song Papa Don’t Preach.
(213/5862) Scraping lyrics for song 4 Minutes.
(215/5862) Scraping lyrics for song I Don’t Give A.
(216/5862) Scraping lyrics for song Express Yourself.
(217/5862) Scraping lyrics for song Frozen.
(218/5862) Scraping lyrics for song Music.
(222/5862) Scraping lyrics for song Give Me All Your Luvin’.
(223/5862) Scraping lyrics for song Sorry.
(224/5862) Scraping lyrics for song Ray of Light.
(225/5862) Scraping lyrics for song Crazy for You.
(226/5862) Scraping lyrics for song Veni Vidi Vici.
(227/5862) Scraping lyrics for song Me Against the Music.
(230/5862) Scraping lyrics for song Holiday.
(231/5862) Scraping lyrics for song Illuminati.
(232/5862) Scraping lyrics for song Live to Tell.
(233/5862) Scraping lyrics for song Erotica.
(235/5862) Scraping lyrics for song Take a Bow.
(

(519/5862) Scraping lyrics for song Could I Have This Kiss Forever.
(520/5862) Scraping lyrics for song I Didn’t Know My Own Strength.
(522/5862) Scraping lyrics for song I Love the Lord.
(523/5862) Scraping lyrics for song My Love is Your Love (Remix).
(526/5862) Scraping lyrics for song The Prince Of Egypt (When You Believe).
(530/5862) Scraping lyrics for song I Believe In You and Me (Film Version).
(531/5862) Scraping lyrics for song Joy to the World.
(544/5862) Scraping lyrics for song All Day.
(545/5862) Scraping lyrics for song FourFiveSeconds.
(546/5862) Scraping lyrics for song Only One.
(548/5862) Scraping lyrics for song Do They Know It’s Christmas?.
(549/5862) Scraping lyrics for song Those Were the Days.
(550/5862) Scraping lyrics for song And I Love Her.
(551/5862) Scraping lyrics for song Can’t Help Falling in Love.
(552/5862) Scraping lyrics for song Jailhouse Rock.
(553/5862) Scraping lyrics for song Love Me Tender.
(554/5862) Scraping lyrics for song Hound Dog.
(555/5

(780/5862) Scraping lyrics for song Takin’ Back My Love.
(781/5862) Scraping lyrics for song Love Sex Magic.
(782/5862) Scraping lyrics for song Read My Lips.
(783/5862) Scraping lyrics for song I Bet (Remix).
(784/5862) Scraping lyrics for song Promise Ring.
(786/5862) Scraping lyrics for song Sophomore.
(790/5862) Scraping lyrics for song Stepped On My J’z.
(792/5862) Scraping lyrics for song Ride (Remix).
(793/5862) Scraping lyrics for song Super Turnt Up.
(794/5862) Scraping lyrics for song DUI.
(795/5862) Scraping lyrics for song Got Me Good.
(796/5862) Scraping lyrics for song The Monster.
(797/5862) Scraping lyrics for song Wild Thoughts.
(798/5862) Scraping lyrics for song LOYALTY..
(799/5862) Scraping lyrics for song Love on the Brain.
(800/5862) Scraping lyrics for song Needed Me.
(801/5862) Scraping lyrics for song Famous.
(802/5862) Scraping lyrics for song Love the Way You Lie.
(803/5862) Scraping lyrics for song Take Care.
(804/5862) Scraping lyrics for song Stay.
(805/58

(1048/5862) Scraping lyrics for song Confessions Part II.
(1049/5862) Scraping lyrics for song Lovers and Friends.
(1050/5862) Scraping lyrics for song U Got It Bad.
(1053/5862) Scraping lyrics for song Good Kisser.
(1054/5862) Scraping lyrics for song Crush.
(1057/5862) Scraping lyrics for song Nice & Slow.
(1059/5862) Scraping lyrics for song Lemme See.
(1060/5862) Scraping lyrics for song U Remind Me.
(1061/5862) Scraping lyrics for song DJ Got Us Fallin’ in Love.
(1062/5862) Scraping lyrics for song Crash.
(1063/5862) Scraping lyrics for song Rest of My Life.
(1064/5862) Scraping lyrics for song Body Language.
(1065/5862) Scraping lyrics for song Love in This Club Part II.
(1066/5862) Scraping lyrics for song Without You.
(1067/5862) Scraping lyrics for song She Came to Give It to You.
(1068/5862) Scraping lyrics for song U Don’t Have to Call.
(1069/5862) Scraping lyrics for song U Know What’s Up (Remix).
(1070/5862) Scraping lyrics for song Love in This Club.
(1071/5862) Scraping 

...

(3993/5862) Scraping lyrics for song Heartbeat Song.
(3994/5862) Scraping lyrics for song Miss Independent.
(3996/5862) Scraping lyrics for song Behind These Hazel Eyes.
(3998/5862) Scraping lyrics for song Heat.
(3999/5862) Scraping lyrics for song My Life Would Suck Without You.
(4002/5862) Scraping lyrics for song Invincible.
(4004/5862) Scraping lyrics for song Catch My Breath.
(4005/5862) Scraping lyrics for song Second Hand Heart.
(4010/5862) Scraping lyrics for song People Like Us.
(4011/5862) Scraping lyrics for song Dark Side.
(4012/5862) Scraping lyrics for song Mr. Know It All.
(4022/5862) Scraping lyrics for song Run Run Run.
(4023/5862) Scraping lyrics for song Don’t You Wanna Stay.
(4024/5862) Scraping lyrics for song Never Again.
(4026/5862) Scraping lyrics for song Wish You Were Here.
(4027/5862) Scraping lyrics for song Time.
(4028/5862) Scraping lyrics for song Comfortably Numb.
(4029/5862) Scraping lyrics for song Another Brick in the Wall, Pt. 2.
(4030/5862) Scrapin

(4209/5862) Scraping lyrics for song Best I Ever Had.
(4210/5862) Scraping lyrics for song All of the Lights.
(4211/5862) Scraping lyrics for song One.
(4212/5862) Scraping lyrics for song Mama Told Me (Not to Come).
(4260/5862) Scraping lyrics for song Nightshift.
(4302/5862) Scraping lyrics for song Roses.
(4303/5862) Scraping lyrics for song Straight Up.
(4304/5862) Scraping lyrics for song Cold Hearted.
(4308/5862) Scraping lyrics for song Rush Rush.
(4310/5862) Scraping lyrics for song Forever Your Girl.
(4352/5862) Scraping lyrics for song Summer of ’69.
(4353/5862) Scraping lyrics for song Heaven.
(4354/5862) Scraping lyrics for song (Everything I Do) I Do It for You.
(4355/5862) Scraping lyrics for song Have You Ever Really Loved a Woman?.
(4356/5862) Scraping lyrics for song Please Forgive Me.
(4375/5862) Scraping lyrics for song (Everything I Do) I Do It For You.
(4383/5862) Scraping lyrics for song You Belong to Me.
(4385/5862) Scraping lyrics for song Run.
(4398/5862) Scrap

(4767/5862) Scraping lyrics for song American Dream Plan B.
(4770/5862) Scraping lyrics for song We Are the World.
(4771/5862) Scraping lyrics for song The Sound of Silence.
(4772/5862) Scraping lyrics for song You Can Call Me Al.
(4774/5862) Scraping lyrics for song Me and Julio Down by the Schoolyard.
(4775/5862) Scraping lyrics for song 50 Ways to Leave Your Lover.
(4776/5862) Scraping lyrics for song Graceland.
(4778/5862) Scraping lyrics for song Bring Dem Things.
(4779/5862) Scraping lyrics for song Oh Cecilia (Breaking My Heart).
(4783/5862) Scraping lyrics for song The Boy in the Bubble.
(4784/5862) Scraping lyrics for song I Know What I Know.
(4785/5862) Scraping lyrics for song Kodachrome.
(4788/5862) Scraping lyrics for song All Delighted People (Original Version).
(4789/5862) Scraping lyrics for song Forever Begins.
(4790/5862) Scraping lyrics for song The Boxer.
(4793/5862) Scraping lyrics for song I Want It That Way.
(4794/5862) Scraping lyrics for song Everybody (Backstr

(5140/5862) Scraping lyrics for song Sleigh Ride.
(5141/5862) Scraping lyrics for song Case of the Fake People.
(5143/5862) Scraping lyrics for song I Miss You So Much.
(5148/5862) Scraping lyrics for song Take Our Time.
(5150/5862) Scraping lyrics for song Switch.
(5151/5862) Scraping lyrics for song Can You Hear Me.
(5153/5862) Scraping lyrics for song Let’s Do it Again.
(5154/5862) Scraping lyrics for song Das Da Way We Like ’Em.
(5155/5862) Scraping lyrics for song This is How it Works.
(5156/5862) Scraping lyrics for song My Life.
(5157/5862) Scraping lyrics for song Dear Lie.
(5158/5862) Scraping lyrics for song Get It Up.
(5159/5862) Scraping lyrics for song Damaged.
(5160/5862) Scraping lyrics for song His Story.
(5166/5862) Scraping lyrics for song You’re Still the One.
(5167/5862) Scraping lyrics for song Man! I Feel like a Woman!.
(5168/5862) Scraping lyrics for song That Don’t Impress Me Much.
(5173/5862) Scraping lyrics for song Forever and for Always.
(5174/5862) Scraping

(5712/5862) Scraping lyrics for song What’s Your Fantasy.
(5713/5862) Scraping lyrics for song Wet the Bed.
(5714/5862) Scraping lyrics for song Tonight (Best You Ever Had).
(5716/5862) Scraping lyrics for song Gossip Folks.
(5717/5862) Scraping lyrics for song We In This Bitch.
(5718/5862) Scraping lyrics for song Holidae In.
(5720/5862) Scraping lyrics for song Same Damn Time (Remix).
(5721/5862) Scraping lyrics for song Ho.
(5722/5862) Scraping lyrics for song Southern Hospitality.
(5723/5862) Scraping lyrics for song Stand Up.
(5725/5862) Scraping lyrics for song Get Back.
(5726/5862) Scraping lyrics for song They Don’t Know (Remix).
(5727/5862) Scraping lyrics for song Welcome to Atlanta.
(5728/5862) Scraping lyrics for song One Minute Man.
(5729/5862) Scraping lyrics for song Welcome to Atlanta (Coast 2 Coast Remix).
(5730/5862) Scraping lyrics for song All Around the World.
(5732/5862) Scraping lyrics for song Act a Fool.
(5734/5862) Scraping lyrics for song Representin.
(5735/5

In [106]:
# Authenticate and set up client
spotify = Spotify()
spotify.authenticate()

In [135]:
# Extract Spotify audio features for each track
feature_keys = {"danceability", "energy", "key", "loudness", "mode", "speechiness", "acousticness", 
                "instrumentalness", "liveness", "valence", "tempo", "time_signature"}
for start in range(0, len(dataset), 100):
    stop = start + 100
    ids = [row["spotify_uri"][len("spotify:track:"):] for row in dataset[start:stop]
           if row["spotify_uri"]]
    try:
        print("({}/{}) Obtaining Spotify audio features for tracks {}-{}.".format(start, len(dataset), start, stop))
        time.sleep(0.05)
        res = spotify.get("audio-features?ids="+ ",".join(ids))
        print(",".join(ids))
        print(res)
        if not res or not "audio_features" in res:
            print("Error in request {}. {}".format(start/100, repr(res)))
            break
        res = res["audio_features"]
        for k, res_row in enumerate(res):
            if not res_row or "uri" not in res_row:
                continue
            features = {k: res_row[k] for k in feature_keys if k in res_row}
            for i, row in enumerate(dataset):
                if not row["spotify_uri"]:
                    continue
                if res_row["uri"] == row["spotify_uri"]:
                    dataset[i].update(features)
                    break
            else:
                print("No matching URI for queried URI found. {}, {}".format(row["spotify_uri"], {r["uri"] for r in res}))
    except Exception as e:
        raise e
        print("Error for Spotify audio features request {}. {}".format(start//100, str(e)))
        break
    break

(0/5862) Obtaining Spotify audio features for tracks 0-100.
(100/5862) Obtaining Spotify audio features for tracks 100-200.
(200/5862) Obtaining Spotify audio features for tracks 200-300.
(300/5862) Obtaining Spotify audio features for tracks 300-400.
(400/5862) Obtaining Spotify audio features for tracks 400-500.
(500/5862) Obtaining Spotify audio features for tracks 500-600.
(600/5862) Obtaining Spotify audio features for tracks 600-700.
(700/5862) Obtaining Spotify audio features for tracks 700-800.
(800/5862) Obtaining Spotify audio features for tracks 800-900.
(900/5862) Obtaining Spotify audio features for tracks 900-1000.
(1000/5862) Obtaining Spotify audio features for tracks 1000-1100.
(1100/5862) Obtaining Spotify audio features for tracks 1100-1200.
(1200/5862) Obtaining Spotify audio features for tracks 1200-1300.
(1300/5862) Obtaining Spotify audio features for tracks 1300-1400.
(1400/5862) Obtaining Spotify audio features for tracks 1400-1500.
(1500/5862) Obtaining Spotif

In [137]:
df = pandas.DataFrame(dataset)
df.to_csv("data/genius_top100.csv")

In [148]:
cleaned_df = df[~df["spotify_uri"].isna() & ~df["tempo"].isna()]
cleaned_df.to_csv("data/genius_top100_clean.csv")
cleaned_df

,genius_id,title,path,artist,artist_genius_id,spotify_uri,lyrics,valence,tempo,time_signature,liveness,instrumentalness,mode,key,acousticness,danceability,loudness,speechiness,energy
0,1575,Let It Be,/The-beatles-let-it-be-lyrics,The Beatles,586,spotify:track:7iN1s7xHE4ifF5povM6A48,\n\n[Verse 1]\nWhen I find myself in times of ...,0.410,143.462,4.0,0.1110,0.000000,1.0,0.0,0.6310,0.443,-8.339,0.0322,0.403
1,2236,Yesterday,/The-beatles-yesterday-lyrics,The Beatles,586,spotify:track:3BQHpFgAp4l80e1XslIjNI,\n\n[Verse 1]\nYesterday\nAll my troubles seem...,0.315,96.529,4.0,0.0886,0.000000,1.0,5.0,0.8790,0.332,-11.830,0.0326,0.179
2,56218,Come Together,/The-beatles-come-together-lyrics,The Beatles,586,spotify:track:2EqlS6tkEnglzr7tkKAAYD,\n\n[Intro]\nShoot me\nShoot me\nShoot me\nSho...,0.187,165.007,4.0,0.0926,0.248000,0.0,9.0,0.0302,0.533,-11.913,0.0393,0.376
3,82381,Hey Jude,/The-beatles-hey-jude-lyrics,The Beatles,586,spotify:track:0aym2LBJBk9DAYuHHutrIl,"\n\n[Verse 1: Paul McCartney]\nHey Jude, don't...",0.532,147.207,4.0,0.0880,0.000014,1.0,10.0,0.0112,0.386,-7.700,0.0261,0.607
4,87564,Something,/The-beatles-something-lyrics,The Beatles,586,spotify:track:0pNeVovbiZHkulpGeOx1Gj,\n\n[Verse 1]\nSomething in the way she moves\...,0.377,133.522,4.0,0.1380,0.000003,1.0,0.0,0.1980,0.396,-10.872,0.0304,0.338
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5817,340023,Remember When,/Alan-jackson-remember-when-lyrics,Alan Jackson,15265,spotify:track:1nCSr7oGeUv7sUEoYuAJFF,\n\n[Verse: 1]\nRemember when I was young and ...,0.267,125.590,4.0,0.1930,0.000319,1.0,2.0,0.7090,0.558,-10.623,0.0295,0.314
5824,332381,As She’s Walking Away,/Zac-brown-band-as-shes-walking-away-lyrics,Alan Jackson,15265,spotify:track:1jxR2ZtJo0O3Zq993R4MV7,\n\n[Verse 1: Zac Brown]\nWe never spoke a wor...,0.334,183.981,4.0,0.1200,0.000000,1.0,3.0,0.4600,0.556,-5.909,0.0310,0.655
5836,75138,Chasin’ That Neon Rainbow,/Alan-jackson-chasin-that-neon-rainbow-lyrics,Alan Jackson,15265,spotify:track:5Wp1Y8jscofORVp0FH0N9l,\n\n[Verse 1]\nDaddy won a radio\nHe tuned it ...,0.831,149.061,4.0,0.0946,0.000000,1.0,7.0,0.2310,0.709,-5.873,0.0338,0.710
5840,2152688,Jim and Jack and Hank,/Alan-jackson-jim-and-jack-and-hank-lyrics,Alan Jackson,15265,spotify:track:4v5tgTrtDPxs0amRB3Njre,"\n\nHey, you were standing by the screen door ...",0.839,123.999,4.0,0.4260,0.000007,1.0,4.0,0.0518,0.677,-7.535,0.0341,0.801
